In [1]:
import os
import random
import torch
from tqdm import tqdm
from torchvision import models as models
import torch.nn as nn
from torchvision import transforms as transforms
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import time

In [2]:
root = '../'
code_folder = 'code'
models_saved_folder = 'saved_models'
all_data_folder = 'all_images'
data_folder = 'images'
train_images_foldr = 'train'
validation_images_foldr = 'val'
test_images_foldr = 'test'
os.makedirs(os.path.join(root, models_saved_folder), exist_ok=True)

In [10]:
SEED = 518
random.seed(SEED)

resize = 224
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transform_tr = transforms.Compose([
    transforms.Resize((int(resize * 1.25), int(resize * 1.25))),
    transforms.RandomRotation(15),
    transforms.CenterCrop(resize),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
transform_val = transforms.Compose([
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)])


train_set = ImageFolder(os.path.join(root, data_folder, train_images_foldr),transform=transform_tr)
val_set = ImageFolder(os.path.join(root, data_folder, validation_images_foldr),transform=transform_val)
test_set = ImageFolder(os.path.join(root, data_folder, test_images_foldr),transform=transform_val)

num_classes = len(train_set.classes)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

batch_size = 1


train_loader = DataLoader(dataset=train_set, batch_size = batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset=val_set, batch_size = batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset=test_set, batch_size = batch_size, shuffle=False, num_workers=0)


def evaluate(model, loader, criterion, device):
    # counter += 1
    valid_running_loss = 0.0
    corrects = 0.0
    total = len(loader.dataset)
    print('Validation...')
    model.eval()
    with torch.no_grad():
        for data, target in loader:
#             start = time.monotonic() 


            data, target = data.to(device), target.to(device)

            output = model(data)

            loss = criterion(output, target)

            _, predicted = torch.max(output.data, 1)
#             end = time.monotonic() 
#             print("Time elapsed during one epoch:", end - start)
            corrects += torch.eq(predicted, target).sum().float().item()
            # print('Val', counter, 'loss', loss.item()m 'acc')

            valid_running_loss += loss.item()
    print('Val loader.sampler:', len(loader.sampler))
    return valid_running_loss / len(loader.sampler), corrects / total


# for i, data in enumerate(train_loader.dataset.imgs):
#     print(i, data)
print('Train:', len(train_loader.sampler))
print('Valid:', len(valid_loader.sampler))
print('Test:', len(test_loader.sampler))

Train: 1146
Valid: 384
Test: 387


In [11]:
res50 = torch.load('materials_shaozuiyu_exp/tut3-model_res50.pt',map_location=torch.device('cpu'))
res101 = torch.load('materials_shaozuiyu_exp/tut3-model_res101.pt',map_location=torch.device('cpu'))
res152 = torch.load('materials_shaozuiyu_exp/tut3-model_res152.pt',map_location=torch.device('cpu'))


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()

criterion = criterion.to(device)

## Training ACC

In [12]:
# res50 training acc
evaluate(model=res50, loader=train_loader, criterion=criterion, device=device)

Validation...
Val loader.sampler: 1146


(0.33476796325598474, 0.9240837696335078)

In [13]:
# res101 training acc
evaluate(model=res101, loader=train_loader, criterion=criterion, device=device)

Validation...
Val loader.sampler: 1146


(0.5495512580321251, 0.8787085514834206)

In [14]:
# res152 training acc
evaluate(model=res152, loader=train_loader, criterion=criterion, device=device)

Validation...
Val loader.sampler: 1146


(0.36977925277610235, 0.9048865619546248)

## Validation ACC

In [15]:
# res50 val acc
evaluate(model=res50, loader=valid_loader, criterion=criterion, device=device)

Validation...
Val loader.sampler: 384


(0.12082504269349552, 0.9661458333333334)

In [16]:
# res101 val acc
evaluate(model=res101, loader=valid_loader, criterion=criterion, device=device)

Validation...
Val loader.sampler: 384


(0.10908404102104896, 0.9739583333333334)

In [17]:
# res152 val acc
evaluate(model=res152, loader=valid_loader, criterion=criterion, device=device)

Validation...
Val loader.sampler: 384


(0.1232516405312191, 0.953125)

## Test ACC

In [18]:
# res50 val acc
evaluate(model=res50, loader=test_loader, criterion=criterion, device=device)

Validation...
Val loader.sampler: 387


(0.11983846340398338, 0.9689922480620154)

In [19]:
# res101 val acc
evaluate(model=res101, loader=test_loader, criterion=criterion, device=device)

Validation...
Val loader.sampler: 387


(0.11974512478503645, 0.9664082687338501)

In [20]:
# res152 val acc
evaluate(model=res152, loader=test_loader, criterion=criterion, device=device)

Validation...
Val loader.sampler: 387


(0.11232502792356433, 0.9689922480620154)